In [16]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt

plt.style.use('ggplot')
# MacOS上画图中文乱码的问题
# https://blog.csdn.net/minixuezhen/article/details/81516949?utm_medium=distribute.pc_relevant.none-task-blog-
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']

%matplotlib inline

import sys
import os

from pprint import pprint

In [17]:
def load_data():
    table_files = """data/附件1 监测点A空气质量预报基础数据.xlsx
data/附件2 监测点B、C空气质量预报基础数据.xlsx
data/附件3 监测点A1、A2、A3空气质量预报基础数据.xlsx""".split("\n")
    tables = [pd.read_excel("../"+i, engine='openpyxl', sheet_name=None) for i in table_files] # 表格全读取
    keys = [list(i.keys()) for i in tables] # 每个表格的sheet list
    return tables, keys
tables, keys = load_data()

In [58]:
class WeatherData(object):
    def __init__(self, tables):
        # 每个表有若干个sheet name
        self.tables = tables
        self.keys = [list(i.keys()) for i in tables]
        
        ## 属性表
        # 逐小时污染物浓度与气象一次预报
        self.columns_prediction_by_hour = list(self.get_data(0, 0).columns)
        # 逐小时污染物浓度与气象实测
        self.columns_polution_weather_by_hour = list(self.get_data(0, 1).columns)
        # 逐日污染物浓度实测
        self.columns_polution_by_day = list(self.get_data(0, 2).columns)
        
    def get_data(self, table_id, sheet_id):
        return self.tables[table_id][self.keys[table_id][sheet_id]]
    
    def analysis_info(self, table_id):
        for i in range(len(self.keys[table_id])):
            print(self.keys[table_id][i])
            print(self.get_data(table_id, i).info())
    def analysis_columns(self, table_id):
        for i in range(len(self.keys[table_id])):
            print(self.keys[table_id][i])
            print(list(self.get_data(table_id, i).columns))   
    
data = WeatherData(tables)
# pprint(data.columns_prediction_by_hour)
# pprint(data.columns_polution_weather_by_hour)
# pprint(data.columns_polution_by_day)

## 分析数值分布

In [59]:
polution_factors = ['SO2监测浓度(μg/m³)', 'NO2监测浓度(μg/m³)', 'PM10监测浓度(μg/m³)', 'PM2.5监测浓度(μg/m³)', 'O3监测浓度(μg/m³)', 'CO监测浓度(mg/m³)']
weather_factors = ['温度(℃)', '湿度(%)', '气压(MBar)', '风速(m/s)', '风向(°)']

df = data.get_data(0,1)
# 分析污染物随着时间的变化
# for pf in polution_factors:
#     df[pf] = pd.to_numeric(df[pf], errors='coerce')
#     df.plot.line(x='监测时间', y=pf, figsize=(10,8 ))